# Datapath example 3

This notebook gives an example of how to build relativley simple data paths.
It assumes that you understand the concepts presented in the example 2
notebook.

## Exampe Data Model
The examples require that you understand a little bit about the example
catalog data model, which is based on the FaceBase project.

### Key tables
- `'dataset'` : represents a unit of data usually a `'study'` or `'experiment'`
- `'sample'` : a biosample
- `'assay'` : a bioassay (typically RNA-seq or ChIP-seq assays)

### Relationships
- `dataset <- sample`: A dataset may have one to many samples. I.e., there 
  is a foreign key reference from sample to dataset.
- `sample <- assay`: A sample may have one to many assays. I.e., there is a
  foreign key reference from assay to sample.

In [1]:
# Import deriva modules
from deriva_common import ErmrestCatalog, get_credential

In [2]:
# Connect with the deriva catalog
protocol = 'https'
hostname = 'www.facebase.org'
catalog_number = 1
credential = None
# If you need to authenticate, use Deriva Auth agent and get the credential
# credential = get_credential(hostname)
catalog = ErmrestCatalog(protocol, hostname, catalog_number, credential)

In [3]:
# Get the path builder interface for this catalog
pb = catalog.getPathBuilder()

## Building a Datapath
We will build a data path by linking tables from the catalog. To make things a little easier we will use python variables to reference the tables. This is not necessary, but simplifies the examples.

In [4]:
dataset = pb.isa.dataset
sample = pb.isa.sample
assay = pb.isa.assay

Build a data path by linking together different tables that are related.
By default, data path returns entities for the _last_ linked entity set
in the path. The following data path will therefore return assays not
datasets.

In [5]:
assays_datapath = dataset.link(sample).link(assay)
print(assays_datapath.uri) # the ERMrest URL

https://www.facebase.org/ermrest/catalog/1/entity/isa:dataset/isa:sample/isa:assay


Get the entity set for this linked data path.

In [6]:
assays_entities = assays_datapath.entities()
len(assays_entities)

171

## Filtering a Datapath

Building off of the path, a filter can be added. In this filter, the assay's
attriburtes may be reference in the expressions. We did not have to split this
step from the prior step.

**Note**:
In these binary comparisons 
the left operand must be an attribute while the right operand must a literal
value.

In [7]:
assays_datapath_filtered = assays_datapath.filter(assay.molecule_type == 'mRNA')
print(assays_datapath_filtered.uri) # the ERMrest URL

https://www.facebase.org/ermrest/catalog/1/entity/isa:dataset/isa:sample/isa:assay/(molecule_type=mRNA)


In [8]:
assays_datapath_filtered_entities = assays_datapath_filtered.entities()
len(assays_datapath_filtered_entities)

6

## Slicing EntitySets
Any entity set can be sliced too.

In [9]:
print (assays_datapath_filtered_entities[2:4])

[{'id': 15, 'dataset': 14068, 'sample': 2, 'replicate': '5', 'sample_composition': 'maxillary process', 'sample_type': 'RNA-seq', 'molecule_type': 'mRNA', 'sample_purification': 'excision', 'markers': 'histology', 'isolation_protocol': '', 'cell_count': 'NA', 'protocol': '', 'pretreatment': 'Trizol', 'fragmentation_method': 'Fragmentation Buffer from Illumina', 'reagent': 'TruSeq stranded total RNA kit', 'reagent_source': 'Illumina', 'reagent_catalog_number': '15032619.0', 'reagent_batch_number': '', 'selection': 'totalRNA', 'library_id': 75, 'alignment_id': 55, 'tracks_id': 35}, {'id': 20, 'dataset': 14068, 'sample': 4, 'replicate': '5', 'sample_composition': 'mandibular process', 'sample_type': 'RNA-seq', 'molecule_type': 'mRNA', 'sample_purification': 'excision', 'markers': 'histology', 'isolation_protocol': '', 'cell_count': 'NA', 'protocol': '', 'pretreatment': 'Trizol', 'fragmentation_method': 'Fragmentation Buffer from Illumina', 'reagent': 'TruSeq stranded total RNA kit', 'reag

Let's see it rendered as a Pandas DataFrame.

In [10]:
assays_datapath_filtered_entities.dataframe

,alignment_id,cell_count,dataset,fragmentation_method,id,isolation_protocol,library_id,markers,molecule_type,pretreatment,...,reagent_batch_number,reagent_catalog_number,reagent_source,replicate,sample,sample_composition,sample_purification,sample_type,selection,tracks_id
0,41,NA,14068,Fragmentation Buffer from Illumina,1,,61,histology,mRNA,Trizol,...,,15032619.0,Illumina,5,1,medial nasal process,excision,RNA-seq,totalRNA,21
1,46,NA,14068,Fragmentation Buffer from Illumina,6,,66,histology,mRNA,Trizol,...,,15032619.0,Illumina,5,3,latero nasal process,excision,RNA-seq,totalRNA,26
2,55,NA,14068,Fragmentation Buffer from Illumina,15,,75,histology,mRNA,Trizol,...,,15032619.0,Illumina,5,2,maxillary process,excision,RNA-seq,totalRNA,35
3,60,NA,14068,Fragmentation Buffer from Illumina,20,,80,histology,mRNA,Trizol,...,,15032619.0,Illumina,5,4,mandibular process,excision,RNA-seq,totalRNA,40
4,62,NA,14130,Fragmentation Buffer from Illumina,25,None,85,Histology,mRNA,Trizol,...,NA,15032619,Illumina,5,1088,face,Excision,RNA-seq,totalRNA,43
5,64,NA,14130,Fragmentation Buffer from Illumina,30,None,90,Histology,mRNA,Trizol,...,None,15032619,Illumina,5,1089,face,Excision,RNA-seq,totalRNA,43


## Datapaths are Immutable Objects
**IMPORTANT**: methods on data paths (i.e., link or filter and others) return new
data path objects. They do not alter the existing data path. Notice here that
getting the entities for the original data path returns the same number as
before. It did not change when the filters were added, as that created and
returned a new data path object.

In [11]:
len(assays_datapath.entities())

171

## Projecting Attributes From Linked Entities

Returning to the initial example, if we want to project additional attributes
from other entities in the data path, we need to be able to reference the
"instances" of linked entity sets at any point in the path. To do so, first
we need to define a few table "aliases" that we can use in the paths.

### Define a table alias
Start by defining an alias for the 'dataset' table. Any table can be aliased.
The argument to the '`as_(...)`' method is a string without special characters
in it.

In [12]:
D = dataset.as_('D')

### Access columns of an aliased table
Like the original table, an alias may be used to reference the columns of the
original table.

In [13]:
D.columns['accession']

Column name: 'accession'	Type: text	Comment: 'None'

Now repeat the path but use the aliased table in place of the table.

In [14]:
datapath = D.link(sample).link(assay)

Project attributes from the last referenced table and any aliased tables.

In [15]:
datapath = datapath.attributes(D.accession, assay.molecule_type, assay.sample_type)
print(datapath.uri)

https://www.facebase.org/ermrest/catalog/1/attribute/D:=isa:dataset/isa:sample/isa:assay/D:accession,molecule_type,sample_type


In [16]:
entities = datapath.entities()
for e in entities[0:10]:
    print(e)

{'accession': 'FB00000806.2', 'molecule_type': 'mRNA', 'sample_type': 'RNA-seq'}
{'accession': 'FB00000806.2', 'molecule_type': 'genomicDNA', 'sample_type': 'Input'}
{'accession': 'FB00000806.2', 'molecule_type': 'genomicDNA', 'sample_type': 'ChIP-seq'}
{'accession': 'FB00000806.2', 'molecule_type': 'genomicDNA', 'sample_type': 'ChIP-seq'}
{'accession': 'FB00000806.2', 'molecule_type': 'genomicDNA', 'sample_type': 'ChIP-seq'}
{'accession': 'FB00000806.2', 'molecule_type': 'mRNA', 'sample_type': 'RNA-seq'}
{'accession': 'FB00000806.2', 'molecule_type': 'genomicDNA', 'sample_type': 'Input'}
{'accession': 'FB00000806.2', 'molecule_type': 'genomicDNA', 'sample_type': 'ChIP-seq'}
{'accession': 'FB00000806.2', 'molecule_type': 'genomicDNA', 'sample_type': 'Input'}
{'accession': 'FB00000806.2', 'molecule_type': 'genomicDNA', 'sample_type': 'ChIP-seq'}


### Alias a table anywhere in the data path
Now define another alias so that sample's columns may be projected as well.

In [17]:
S = sample.as_('S')

This is an all new datapath instance. When linking the samples table we will
need to first indicate which table is being linked, and then what alias to
link it "as". This is similar in spirit to the SQL concept of joining tables
and renaming them "as" a given table instance name.

In [18]:
datapath = D.link(sample, as_=S).link(assay).attributes(D.accession, S.stage, assay.sample_type)
print(datapath.uri)

https://www.facebase.org/ermrest/catalog/1/attribute/D:=isa:dataset/S:=isa:sample/isa:assay/D:accession,S:stage,sample_type


In [19]:
for e in datapath.entities(limit=5):
    print(e)

{'accession': 'FB00000806.2', 'stage': 30, 'sample_type': 'RNA-seq'}
{'accession': 'FB00000806.2', 'stage': 30, 'sample_type': 'Input'}
{'accession': 'FB00000806.2', 'stage': 30, 'sample_type': 'ChIP-seq'}
{'accession': 'FB00000806.2', 'stage': 30, 'sample_type': 'ChIP-seq'}
{'accession': 'FB00000806.2', 'stage': 30, 'sample_type': 'ChIP-seq'}
